In [2]:
from tensorflow.keras.applications import VGG16
from tensorflow.keras import models
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam

In [5]:
import cv2
import numpy as np
import os

print("[INFO] loading face detection model...")
prototxt_path = "deploy.prototxt.txt"  
model_path = "res10_300x300_ssd_iter_140000.caffemodel"  
face_net = cv2.dnn.readNetFromCaffe(prototxt_path, model_path)

train_test_base_dir = 'train_test'  
cropped_images_base_dir = 'cropped_images_1'  


os.makedirs(cropped_images_base_dir, exist_ok=True)

def detect_and_crop_faces(image_path, output_path):
    image = cv2.imread(image_path)
    if image is None:
        print(f"Warning: Could not read image {image_path}")
        return
    (h, w) = image.shape[:2]
    blob = cv2.dnn.blobFromImage(cv2.resize(image, (300, 300)), 1.0, (300, 300), (104.0, 177.0, 123.0))
    face_net.setInput(blob)
    detections = face_net.forward()

    faces_found = False  

    for i in range(0, detections.shape[2]):
        confidence = detections[0, 0, i, 2]
        if confidence > 0.5:
            box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
            (startX, startY, endX, endY) = box.astype("int")
            if startX >= 0 and startY >= 0 and endX <= w and endY <= h:
                face = image[startY:endY, startX:endX]
                if face.size > 0:
                    cv2.imwrite(output_path, face)
                    faces_found = True
                    break 
    if not faces_found:
        print(f"No valid faces found in image {image_path}")

for main_dir_name in os.listdir(train_test_base_dir):
    main_dir_path = os.path.join(train_test_base_dir, main_dir_name)
    if os.path.isdir(main_dir_path):
        cropped_main_dir_path = os.path.join(cropped_images_base_dir, main_dir_name)
        os.makedirs(cropped_main_dir_path, exist_ok=True)

        for person_name in os.listdir(main_dir_path):
            person_dir = os.path.join(main_dir_path, person_name)
            if os.path.isdir(person_dir):
                cropped_faces_person_dir = os.path.join(cropped_main_dir_path, person_name)
                os.makedirs(cropped_faces_person_dir, exist_ok=True)

                for image_file in os.listdir(person_dir):
                    if image_file.lower().endswith((".jpg", ".jpeg", ".png")):
                        image_path = os.path.join(person_dir, image_file)
                        output_path = os.path.join(cropped_faces_person_dir, image_file)  
                        detect_and_crop_faces(image_path, output_path)

print("Face cropping process is complete.")


[INFO] loading face detection model...
Face cropping process is complete.


In [12]:
from tensorflow.keras.applications import VGG16
from tensorflow.keras import models, layers
from tensorflow.keras.optimizers import Adam

base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

for layer in base_model.layers:
    layer.trainable = False


model = models.Sequential()
model.add(base_model)
model.add(layers.Flatten())
model.add(layers.Dense(1024, activation='relu'))
model.add(layers.BatchNormalization())  
model.add(layers.Dropout(0.5))  
model.add(layers.Dense(5, activation='softmax'))

model.compile(optimizer=Adam(lr=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

model.summary()


Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, 7, 7, 512)         14714688  
                                                                 
 flatten_2 (Flatten)         (None, 25088)             0         
                                                                 
 dense_4 (Dense)             (None, 1024)              25691136  
                                                                 
 batch_normalization (Batch  (None, 1024)              4096      
 Normalization)                                                  
                                                                 
 dropout_2 (Dropout)         (None, 1024)              0         
                                                                 
 dense_5 (Dense)             (None, 5)                 5125      
                                                      

In [13]:
model.compile(optimizer=Adam(learning_rate=0.0001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [35]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np

train_dir = 'cropped_images/train'
valid_dir = 'cropped_images/valid'


train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,  
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True, 
    brightness_range=[0.5, 1.5],  
    fill_mode='nearest' 
)

valid_datagen = ImageDataGenerator(
    rescale=1./255  
)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

validation_generator = valid_datagen.flow_from_directory(
    valid_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)


Found 4000 images belonging to 5 classes.
Found 1000 images belonging to 5 classes.


In [36]:
from tensorflow.keras.callbacks import EarlyStopping

early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // validation_generator.batch_size,
    epochs=20,
    callbacks=[early_stopping]  
)


Epoch 1/20
125/125 [==============================] - 664s 5s/step - loss: 1.6064 - accuracy: 0.2175 - val_loss: 0.1512 - val_accuracy: 0.9587
Epoch 2/20
125/125 [==============================] - 655s 5s/step - loss: 1.5997 - accuracy: 0.2345 - val_loss: 0.3228 - val_accuracy: 0.9365
Epoch 3/20
125/125 [==============================] - 681s 5s/step - loss: 1.5922 - accuracy: 0.2407 - val_loss: 0.4637 - val_accuracy: 0.9425
Epoch 4/20
 66/125 [==============>...............] - ETA: 4:31 - loss: 1.5851 - accuracy: 0.2363

In [17]:
model.save('face_recognition_vgg16_model_1.h5')

C:\Users\noble\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [18]:
import numpy as np
from sklearn.metrics import confusion_matrix, classification_report
class_names = {0: 'Jeswin', 1: 'Joshua', 2: 'Manoah', 3: 'Nikhil', 4: 'Rovin'}

y_true = validation_generator.classes

y_pred = model.predict(validation_generator)

y_pred_classes = np.argmax(y_pred, axis=1)

conf_matrix = confusion_matrix(y_true, y_pred_classes)

class_report = classification_report(y_true, y_pred_classes, target_names=class_names.values())

class_accuracy = conf_matrix.diagonal() / conf_matrix.sum(axis=1)

print("Confusion Matrix:")
print(conf_matrix)
print("\nClassification Report:")
print(class_report)
print("\nAccuracy for each class:")
for class_name, accuracy in zip(class_names.values(), class_accuracy):
    print(f"{class_name}: {accuracy:.2f}")


32/32 [==============================] - 81s 3s/step
Confusion Matrix:
[[36 39 39 49 37]
 [41 33 41 39 46]
 [35 44 42 37 42]
 [46 36 40 38 40]
 [41 47 40 37 35]]

Classification Report:
              precision    recall  f1-score   support

      Jeswin       0.18      0.18      0.18       200
      Joshua       0.17      0.17      0.17       200
      Manoah       0.21      0.21      0.21       200
      Nikhil       0.19      0.19      0.19       200
       Rovin       0.17      0.17      0.17       200

    accuracy                           0.18      1000
   macro avg       0.18      0.18      0.18      1000
weighted avg       0.18      0.18      0.18      1000


Accuracy for each class:
Jeswin: 0.18
Joshua: 0.17
Manoah: 0.21
Nikhil: 0.19
Rovin: 0.17


In [8]:
import cv2
import numpy as np
import os
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import load_model

model = load_model('face_recognition_vgg16_model_1.h5')

class_names = {0: 'Jeswin', 1: 'Joshua', 2: 'Manoah', 3: 'Nikhil', 4: 'Rovin'}

images_directory = 'cropped_images/valid/Manoah'

class_counts = {class_name: 0 for class_name in class_names.values()}

for filename in os.listdir(images_directory):
    if filename.lower().endswith(('.jpg', '.jpeg', '.png')):
        image_path = os.path.join(images_directory, filename)
        
        img = cv2.imread(image_path)
        img_resized = cv2.resize(img, (224, 224))  
        img_array = image.img_to_array(img_resized)  
        img_array_expanded_dims = np.expand_dims(img_array, axis=0)  
        img_preprocessed = img_array_expanded_dims / 255.0  
        
        predictions = model.predict(img_preprocessed)
        
        predicted_class_index = np.argmax(predictions, axis=1)[0]
        predicted_class_name = class_names[predicted_class_index]
        
        class_counts[predicted_class_name] += 1
        
        print(f"{filename}: Predicted class - {predicted_class_name}")

print("\nCounts of predictions for each class:")
for class_name, count in class_counts.items():
    print(f"{class_name}: {count}")


1/1 [==============================] - 0s 336ms/step
frame_02f44eda-af67-4b11-82a9-f58ffbee6a0e.jpg: Predicted class - Manoah
1/1 [==============================] - 0s 130ms/step
frame_039c536c-dcac-4b1d-b68e-8c4320f63fdb.jpg: Predicted class - Manoah
1/1 [==============================] - 0s 134ms/step
frame_04a7afd5-4788-4f48-a0e9-8ace46b7412b.jpg: Predicted class - Manoah
1/1 [==============================] - 0s 115ms/step
frame_04bdce2d-a319-4a18-89b9-f48a5e0e0d2c.jpg: Predicted class - Manoah
1/1 [==============================] - 0s 134ms/step
frame_06d69303-de8c-4d71-a1cc-f19c68e73081.jpg: Predicted class - Manoah
1/1 [==============================] - 0s 129ms/step
frame_0b135555-905d-491b-99e8-311fad9089dd.jpg: Predicted class - Manoah
1/1 [==============================] - 0s 133ms/step
frame_0ba353ca-ab3f-4ae4-a739-4fe17ff51309.jpg: Predicted class - Manoah
1/1 [==============================] - 0s 131ms/step
frame_0c5a0a6e-a27b-434c-99be-7f4790f69474.jpg: Predicted class -

In [3]:
import cv2
import numpy as np
import os
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import load_model
import json

model = load_model('face_recognition_vgg16_model_1.h5')

class_names = {0: 'Jeswin', 1: 'Joshua', 2: 'Manoah', 3: 'Nikhil', 4: 'Rovin'}

emotions_occurrences = {}

main_directory = 'parent'

for subdir in os.listdir(main_directory):
    subdir_path = os.path.join(main_directory, subdir)
    if os.path.isdir(subdir_path):  
        emotions_occurrences[subdir] = {'Jeswin': 0, 'Joshua': 0, 'Manoah': 0, 'Nikhil': 0, 'Rovin': 0}
        
        for filename in os.listdir(subdir_path):
            if filename.lower().endswith((".jpg", ".jpeg", ".png")):  
                img_path = os.path.join(subdir_path, filename)
                img = cv2.imread(img_path)
                if img is not None:
                    img_resized = cv2.resize(img, (224, 224))
                    img_array = image.img_to_array(img_resized)
                    img_array_expanded_dims = np.expand_dims(img_array, axis=0)
                    img_preprocessed = img_array_expanded_dims / 255. 

                    prediction = model.predict(img_preprocessed)
                    predicted_class_index = np.argmax(prediction, axis=1)[0]
                    predicted_class_name = class_names[predicted_class_index]

                    emotions_occurrences[subdir][predicted_class_name] += 1

print("Total occurrences per emotion in the directory:", emotions_occurrences)

with open('emotion_occurrences.json', 'w') as json_file:
    json.dump(emotions_occurrences, json_file, indent=4)


1/1 [==============================] - 0s 136ms/step
Total occurrences per emotion in the directory: {'Anger': {'Jeswin': 0, 'Joshua': 0, 'Manoah': 1, 'Nikhil': 0, 'Rovin': 0}, 'Angry': {'Jeswin': 0, 'Joshua': 0, 'Manoah': 0, 'Nikhil': 0, 'Rovin': 0}, 'Disgust': {'Jeswin': 0, 'Joshua': 0, 'Manoah': 24, 'Nikhil': 0, 'Rovin': 0}, 'Disgusted': {'Jeswin': 0, 'Joshua': 0, 'Manoah': 0, 'Nikhil': 0, 'Rovin': 0}, 'Fear': {'Jeswin': 0, 'Joshua': 0, 'Manoah': 0, 'Nikhil': 0, 'Rovin': 0}, 'Fearful': {'Jeswin': 0, 'Joshua': 0, 'Manoah': 4, 'Nikhil': 0, 'Rovin': 0}, 'Happiness': {'Jeswin': 0, 'Joshua': 0, 'Manoah': 18, 'Nikhil': 0, 'Rovin': 0}, 'Happy': {'Jeswin': 0, 'Joshua': 0, 'Manoah': 36, 'Nikhil': 0, 'Rovin': 0}, 'Neutral': {'Jeswin': 0, 'Joshua': 0, 'Manoah': 7, 'Nikhil': 0, 'Rovin': 0}, 'Sad': {'Jeswin': 0, 'Joshua': 0, 'Manoah': 0, 'Nikhil': 0, 'Rovin': 0}, 'Sadness': {'Jeswin': 0, 'Joshua': 0, 'Manoah': 36, 'Nikhil': 0, 'Rovin': 0}, 'Surprise': {'Jeswin': 0, 'Joshua': 0, 'Manoah': 11, 'Ni

In [7]:
import cv2
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import load_model
model = load_model('face_recognition_vgg16_model_1.h5')
class_names = {0: 'Jeswin', 1: 'Joshua', 2: 'Manoah', 3: 'Nikhil', 4: 'Rovin'}

img = cv2.imread('parent/Sad/face_501_269.jpg')
img_resized = cv2.resize(img, (224, 224))
img_array = image.img_to_array(img_resized)
img_array_expanded_dims = np.expand_dims(img_array, axis=0)
img_preprocessed = img_array_expanded_dims / 255.  

prediction = model.predict(img_preprocessed)
predicted_class_index = np.argmax(prediction, axis=1)[0]
predicted_class_name = class_names[predicted_class_index]
print(predicted_class_name)

1/1 [==============================] - 0s 297ms/step
Rovin


In [1]:
from tensorflow.keras.models import load_model

# Replace 'path/to/your/model.h5' with the actual path to your saved model
model = load_model('face_recognition_vgg16_model_1.h5')
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Assuming you have a validation data directory and preprocessing steps defined
validation_datagen = ImageDataGenerator(rescale=1./255)  # Example preprocessing step

validation_generator = validation_datagen.flow_from_directory(
    'cropped_images/valid',
    target_size=(224, 224),  # Assuming this is the target size you trained with
    batch_size=32,  # Or your batch size
    class_mode='categorical',  # Assuming you're doing categorical classification
    shuffle=False  # Important for evaluation to not shuffle the data
)
import numpy as np
from sklearn.metrics import confusion_matrix, classification_report

# Your class names
class_names = {0: 'Jeswin', 1: 'Joshua', 2: 'Manoah', 3: 'Nikhil', 4: 'Rovin'}

# Obtain true labels and predictions
y_true = validation_generator.classes
y_pred = model.predict(validation_generator)
y_pred_classes = np.argmax(y_pred, axis=1)

# Generate confusion matrix and classification report
conf_matrix = confusion_matrix(y_true, y_pred_classes)
class_report = classification_report(y_true, y_pred_classes, target_names=class_names.values())

# Calculate class-wise accuracy
class_accuracy = conf_matrix.diagonal() / conf_matrix.sum(axis=1)

# Print the results
print("Confusion Matrix:")
print(conf_matrix)
print("\nClassification Report:")
print(class_report)
print("\nAccuracy for each class:")
for class_name, accuracy in zip(class_names.values(), class_accuracy):
    print(f"{class_name}: {accuracy:.2f}")





Found 1000 images belonging to 5 classes.
32/32 [==============================] - 165s 5s/step
Confusion Matrix:
[[198   0   2   0   0]
 [  1 199   0   0   0]
 [  0   0 200   0   0]
 [  0   0   0 200   0]
 [  0   0   0   0 200]]

Classification Report:
              precision    recall  f1-score   support

      Jeswin       0.99      0.99      0.99       200
      Joshua       1.00      0.99      1.00       200
      Manoah       0.99      1.00      1.00       200
      Nikhil       1.00      1.00      1.00       200
       Rovin       1.00      1.00      1.00       200

    accuracy                           1.00      1000
   macro avg       1.00      1.00      1.00      1000
weighted avg       1.00      1.00      1.00      1000


Accuracy for each class:
Jeswin: 0.99
Joshua: 0.99
Manoah: 1.00
Nikhil: 1.00
Rovin: 1.00
